<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>LSTM - Clase 5 - V3.0</strong></p> 

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt


In [ ]:
# Lee el archivo como un DataFrame
data_full  = pd.read_csv('data/sell-in.csv', delimiter='\t')
df_pid_validos  = pd.read_csv('data/productos_a_predecir.txt')

# Ajustar el ancho máximo de las columnas
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho máximo de la visualización
pd.set_option('display.width', 1000)

# Muestra las primeras filas del DataFrame
print(data_full.head())
print(df_pid_validos.head())

In [ ]:
# Armado de la lista de productos validos a predecir para el periodo
product_ids_validos = df_pid_validos['product_id'].tolist()

# Ver cuántos registros tiene la lista
num_registros = len(product_ids_validos)
print(f"La lista product_ids_validos tiene {num_registros} registros.")

In [ ]:
# Filtrar el DataFrame original para quedarse solo con los product_id válidos
data = data_full[data_full['product_id'].isin(product_ids_validos)]

# Ver cuántos registros tiene el DataFrame filtrado
num_registros_filtrados = len(data)
print(f"El DataFrame filtrado tiene {num_registros_filtrados} registros.")

# Verificar que todos los product_id en el DataFrame filtrado están en la lista de productos válidos
productos_unicos_filtrados = data['product_id'].unique()
productos_invalidos = [pid for pid in productos_unicos_filtrados if pid not in product_ids_validos]

if len(productos_invalidos) == 0:
    print("Todos los product_id en el DataFrame filtrado son válidos.")
else:
    print(f"Se encontraron productos no válidos en el DataFrame filtrado: {productos_invalidos}")

# (Opcional) Ver cuántos registros hay por cada product_id
registros_por_producto = data['product_id'].value_counts()
print("Número de registros por product_id en el DataFrame filtrado:")
print(registros_por_producto)

In [ ]:
# Filtrar los datos para el período especificado
data['periodo'] = data['periodo'].astype(str)
data = data[(data['periodo'] >= '201801') & (data['periodo'] <= '201812')]

# Agrupar por product_id
products = data['product_id'].unique()

# Función para crear las secuencias de datos
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Diccionario para almacenar los resultados
results = {'product_id': [], 'tn': []}

for i, product_id in enumerate(products):
    print(f"Procesando product_id: {product_id} ({i + 1} de {len(products)})")
    
    product_data = data[data['product_id'] == product_id]['tn'].values.reshape(-1, 1)
    
    # Estandarizar los datos
    scaler = StandardScaler()
    product_data = scaler.fit_transform(product_data)
    
    # Crear secuencias de datos
    seq_length = 12
    X, y = create_sequences(product_data, seq_length)
    
    # Dividir los datos en entrenamiento y prueba
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    # Crear el modelo LSTM
    model = Sequential()
    model.add(Input(shape=(seq_length, 1)))  # Utilizar Input en lugar de input_shape
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    
    # Entrenar el modelo
    print(f"Entrenando modelo para product_id: {product_id}")
    model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)
    
    # Realizar predicciones
    predictions = model.predict(X_test)
    
    # Revertir la estandarización
    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test)
    
    # Almacenar los resultados
    results['product_id'].append(product_id)
    results['tn'].append(predictions[-1][0])
    print(f"Predicción para product_id {product_id}: {predictions[-1][0]}")

# Convertir los resultados a un DataFrame y guardarlos en un CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_tn.v3.csv', index=False)

print("Proceso completado. Los resultados se han guardado en 'predicted_tn.v3.csv'.")

<a id = "1.0"></a>
<p style="font-size:25px;color:#34656d;font-family:'Comic Sans MS';text-align:center;border-radius:5px;">
<strong>______________________</strong></p> 